# Hello Evolution: A baseline GA

This is a demonstration of an GA for automatic programming of a Synthesizer. We assume a synthesizer with the following parameters

- `osc_1` : The shape of the first oscillator (eg Sine, Sawtooth, Triangle, Square etc)
- `amp_1` : The amplitude of the first oscillator in the range 0.3-0.7.
- `osc_2` : The shape of the second oscillator (eg Sine, Sawtooth, Triangle, Square etc)
- `amp_2` : The amplitude of the second oscillator in the range 0.3-0.7.

It holds that 

`amp_2` = 1-`amp_1`

so in future experiments, the `amp_2` parameter can be omitted.

First let's import all necessary modules. For this experiment we use the [deap](https://github.com/DEAP/deap) library.

In [1]:
from deap import creator, base, tools, algorithms
import numpy as np
import random
import synth
import librosa

Now let's define the parameters that we are going to use and the osc shape values.

In [2]:
labels = ['osc_1', 'amp_1', 'osc_2', 'amp_2']
SHAPES = list(synth.osc_1_options.keys())
print(SHAPES)

['Sine', 'Triangle', 'Square', 'SquareH', 'Sawtooth', 'Pulse', 'Semicircle']


Now let's define the target sound. The ground truth parameters are:
```
osc_1 = 'Sine'
amp_1 = 0.4
osc_2 = 'Sawtooth'
amp_2 = 0.6
```

In [3]:
mysynth = synth.Synth()
target_features = mysynth.set_parameters(osc_1='Sine',
                                         amp_1=0.4,
                                         osc_2='Sawtooth',
                                         amp_2=0.6)
soundarray = mysynth.get_sound_array()

Time for the definition of the feature extraction, the mate and mutate operators and the evaluation function.

In [4]:
def features(sound_array):
    sr = 44100
    centroid = librosa.feature.spectral_centroid(sound_array, sr)[0]
    bandwidth = librosa.feature.spectral_bandwidth(sound_array, sr)[0]
    flatness = librosa.feature.spectral_flatness(sound_array)[0]
    rolloff = librosa.feature.spectral_rolloff(sound_array)[0]
    result = np.array([centroid, bandwidth, flatness, rolloff])
    return result.flatten()

# Target feature vector
target_f = features(soundarray)

def custom_evaluate(params):
    """Sum of squared error evaluation function"""
    mysynth = synth.Synth()
    params = dict(zip(labels,params))
    mysynth.set_parameters(**params)
    soundarray = mysynth.get_sound_array()
    f_vector = features(soundarray)
    return np.sum(np.square(f_vector - target_f)),

def custom_mate(ind1, ind2, indpb):
    """Mating function. This is probably incorrect"""
    for key in range(2):
        if random.random() < indpb:
            ind1[key], ind2[key] = ind2[key], ind1[key]
    return ind1, ind2

def custom_mutate(individual, indpb):
    """Mutation function. We need to check the probability before mutating I think"""
    i = random.choice([0,1,2,3])
    if i == 0 or i == 2:
        individual[i] = random.choice(SHAPES)
    else:
        individual[i] = np.random.uniform(low=0.3, high=0.7, size=(1,))[0]
    
    return individual,

Now let's define the GA algorithm with the help of DEAP

In [5]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_shape", lambda : random.choice(SHAPES))
toolbox.register("attr_amp", lambda : np.random.uniform(low=0.3, high=0.7, size=(1,))[0])
toolbox.register("individual",
                 tools.initCycle,
                 creator.Individual,
                 (toolbox.attr_shape, toolbox.attr_amp),
                 2)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", custom_evaluate) 
toolbox.register("mate", custom_mate, indpb=0.5)
toolbox.register("mutate", custom_mutate, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

Time to run the algorithm. We pick 10 generations of 30 individuals.
But first, let's see the initial generation.

In [6]:
population = toolbox.population(n=30)
tools.selBest(population, k=30)

[['Semicircle', 0.3248762530116565, 'Sawtooth', 0.340121365621864],
 ['Triangle', 0.4159077405545425, 'Triangle', 0.4938322927210526],
 ['Sawtooth', 0.48122108100340144, 'Pulse', 0.5133189396914227],
 ['SquareH', 0.31868635059871125, 'Sine', 0.6874827972477514],
 ['Pulse', 0.4728196754694046, 'Square', 0.404995960506596],
 ['Semicircle', 0.44608800702040186, 'SquareH', 0.5252303102484442],
 ['Triangle', 0.6611685619481514, 'SquareH', 0.6082406480464767],
 ['Sine', 0.3703478168524261, 'Square', 0.6827191880084751],
 ['Square', 0.4902504882587708, 'Sine', 0.33025441176872355],
 ['Sawtooth', 0.69757562661747, 'Pulse', 0.6501433366097406],
 ['Semicircle', 0.3445848125637166, 'Sine', 0.435916235686384],
 ['Semicircle', 0.683741584365207, 'Sine', 0.4390566774623891],
 ['Square', 0.532831851216794, 'Triangle', 0.38684364689471046],
 ['Triangle', 0.6812800201681788, 'Square', 0.5054556176651712],
 ['SquareH', 0.3019743305201912, 'SquareH', 0.4119683834129382],
 ['SquareH', 0.6799504021348093, 

In [7]:
NGEN=10
for gen in range(NGEN):
    print(gen)
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))

0
1
2
3
4
5
6
7
8
9


The two best results are:

In [9]:
tools.selBest(population, k=2)

[['Sine', 0.3703478168524261, 'Sawtooth', 0.5536177238581826],
 ['Sine', 0.3703478168524261, 'Sawtooth', 0.5536177238581826]]